In [ ]:
import torch
import torch.nn as nn
from torch.nn import functional as F
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)
block_size = 8
batch_size = 4

cuda


In [2]:
with open('wizard_of_oz.txt', 'r', encoding='utf-8') as f:
    text = f.read()
chars = sorted(set(text))
vocab_size = len(text)

In [3]:
string_to_int = {ch:i for i,ch in enumerate(chars)}
int_to_string = {i:ch for i,ch in enumerate(chars)}

encode = lambda s: [string_to_int[c] for c in s]
decode = lambda l: ''.join(int_to_string[i] for i in l)

data = torch.tensor(encode(text), dtype=torch.long)

In [4]:
n = int(0.8 * len(data))
train_data = data[:n]
val_data = data[n:]

def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size, ))
    print(ix)
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

x, y = get_batch('train')
print('inputs: ', x)
print('targets: ', y)

tensor([200417,   6713, 148166,  48895])
inputs:  tensor([[61,  1, 70, 78, 60, 65,  1, 75],
        [16,  0,  0,  1,  1, 18, 16,  1],
        [72,  1, 68, 62, 62, 73,  1, 64],
        [ 0,  0,  3, 48, 65, 66, 76,  1]], device='cuda:0')
targets:  tensor([[ 1, 70, 78, 60, 65,  1, 75, 62],
        [ 0,  0,  1,  1, 18, 16,  1,  1],
        [ 1, 68, 62, 62, 73,  1, 64, 72],
        [ 0,  3, 48, 65, 66, 76,  1, 66]], device='cuda:0')


In [ ]:
class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)
    
    def forward(self, index, targets):
        logits = self.token_embedding_table(index)
        return logits